# Regularization and Model Selection

## Remember the 3 integral steps!


1.   Model construction
2.   Model usage
3.   Model selection



##We've talked about step 1 (Model construction)...

We use our training data to find our estimates

##And we've talked about step 2 (Model usage)...

We use our data against test data

##...So what about model selection?

How do we know which model will be the best fit for our data? How do we prevent overfitting?

## Variable selection

We can use stepwise variable selection to select the best predictors using stepwise variable selection & cross validation!

In stepwise variable selection we iteratively find the optimal set of predictors by slowly building up how many predictors we are using.

1. Start with no predictors
2. Choose the predictor with the highest R^2 value (or other metric)
3. Select the next predictor that increases the R^2 value until no new candidates remain that will increase the R^2 (again, or other metric we are using the evaluate our model's accuracy)



#Variable (forward) selection example!

Download the .csv file from this link here: https://drive.google.com/file/d/10UeKpTSuqPnTeydHmqrSzYGbvVoCX4zF/view?usp=sharing

Import your libraries...

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

Before going forward, let's uninstall and reinstall scikit learn
This is because colab's default sklearn is version 0.22.2, and we want 0.24.2 for this since 0.22.2 does not have sequential feature selection!

In [ ]:
#only do this once - don't re-run it after restarting your runtime

!pip uninstall scikit-learn -y

!pip install -U scikit-learn

import sklearn

sklearn.__version__

Load our dataset...

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/student_scores_extended.csv')

Explore our dataset...

In [ ]:
dataset.head(10)

In [ ]:
dataset.info()

Let's plot each attribute and see what it looks like!

In [ ]:
attributes = ['Previous score', 'Prereqs taken', 'ID number', 'Hours']

for property in attributes:
  dataset.plot(x=property, y='Scores', style='o')
  plt.title(property + ' vs Percentage')
  plt.xlabel(property)
  plt.ylabel('Percentage Score')
  plt.show()

**What do you notice about these plots already?**

Now let's do forward selection! First, get out training and testing data

In [ ]:
from sklearn.model_selection import train_test_split # remember this from yesterday?

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

Next, let's decide what model we are using (let's go with linear regression!)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

Now, let's implement our step forward feature selector

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

selector = SequentialFeatureSelector(
    lr,
    n_features_to_select=None,
    direction='forward',
    scoring='r2',
    cv = 5
)

selector.fit(X_train,y_train)

Now, let's print out which columns (features) were chosen as the optimal featuers!

In [ ]:
selector.get_support()

This means it chose 1,3 as our optimal features! `n_features_to_select=None` means it chooses half the features by default

**Do those results make sense to you?**

You can also do `selector.transform(X)` to directly get a transformed version without the other features

In [ ]:
selector.transform(X_train).shape

# Cross Validation

We want to prevent overfitting! So we can cross validate our models onto various data sets and average our performance. There are various methods explained more in lecture

*   K-fold
*   Leave-One-Out

Let's take a look at a dummy example...




In [ ]:
from sklearn.model_selection import KFold
import numpy as np

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])

kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]

#Regularization

We don't want to deal with extremes/outliers that may skew our data. Regularization helps us modify our loss function (how much our predictions differed from the actual values). We can use two types of regularization methods. This also helps with overfitting!

1.   L1 - Lasso Regularization
2.   L2 - Ridge Regularization

Let's use MSE (Mean Squared Error) as our loss function



##Lasso Regularization (L1)

AKA Least Absolute Shrinkage and Selection Operator

We want to reduce overfitting and control our regularization parameter lambda. Lasso regularization takes the magnitude of our lambda into account by adding a penalty to our loss function which is the absolute value of the magnitude of the coefficient. This method basically shrinks our data until it reaches some middle point. Beware it can lead to a sparse model (small number of coefficients)!

Let's look at an example in code!

In [ ]:
from sklearn.linear_model import Lasso
import numpy as np

In [ ]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [ ]:
l1 = Lasso(alpha=0.1)
l1.fit(X_train, y_train)
l1.score(X_test, y_test)

In [ ]:
print(l1.coef_) # prints the parameters/coefficients for each of our features that will reduce our loss function
print(l1.intercept_) # intercept for our loss function

Note: Lasso regularization tends to make coefficients zero which ends up reducing features. So it can help us with model/feature selection like we just explored!

#Ridge Regularization (L2)

Similar to lasso except lasso tends to make its coefficients zero whereas ridge will never do that. In ridge, our penalty is the square of our coefficient.

Let's look at an example in code!

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
l2 = Ridge(alpha=0.1)
l2.fit(X_train, y_train)
l2.score(X_test, y_test)

In [ ]:
print(l2.coef_)
print(l2.intercept_)

**Let's compare the results from Ridge to our results from Lasso!**